# ${Aflow}$ Implementation experiments

## Source
* https://arxiv.org/pdf/2410.10762

## Notes
* Use gpt-4o-mini as executor/workflow constructor (executor/optimizer)
* backprop through modifications (?) that are descriptions for the executor to implement onto the parent
     * therefore need a way to modify the parent via text

## Algorithm
* Split data into 20% val/80% training
* $async$ Get scores for initial workflow on val data
* Pare down val data to examples with high score variance (above threshold)
* set $experiences = []$
* set $allresults = []$
* set $topkW = []$
* set $W^* = None$
* set $topkScores = []$
* set $bestScore = 0$

* for each training step,
    * set parent as initial workflow for first step or select parent from last training step results using subroutine ${SelectParent}$
    * get the optimizer context (parent workflow + last step experiences)
    * use the optimizer with the context and set of available operators to improve the parent workflow and generate $W_{round}$ and the ${modification}$ it's based on
    * set $evalresults=[]$
    * $async$ for each of 5 rounds
        * execute $W_{round}$ on the val data and get ${score}/{cost}$ from evaluator ${E} / {G}$
        * append the ${score},{cost}$ to $evalresults$
    * append $evalresults$ to $allresults$
    * calculate the average $score$ for the 5 rounds from $evalresults$
    * create a new experience with the parent workflow, the modification to produce $W_{round}$, and the average $score$
    * append the experience to $experiences$ 
    * if the average $score$ is higher than the $bestScore$, update $W^* = W_{round}$ and $bestScore = $ average $score$
    * add $W_{round}$ to $topkW$ and average $score$ to $topkScores$ if $topkW$ has fewer than $k$ elements or average $score$ is greater than the lowest value in $topkScores$
    * if $topkScores$ hasn't changed in specified (n) number of rounds, stop early and return $W^*$
* return $W^*$
    
### Algorithm Unknowns
* how to get evaluator G/E?
* how do initial scores get computed? are there multiple rounds of scoring per example in order to generate variance samples?
* how does the optimizer generate new workflows from previous samples (experiences)?


In [1]:
from math import e
from random import choices,seed,random
from uuid import uuid5 as uuid
from uuid import NAMESPACE_DNS
from copy import deepcopy
from pydantic import BaseModel,conint
from tqdm.notebook import tqdm
from tqdm.asyncio import tqdm as tqdm_asyncio
import asyncio
import nest_asyncio
nest_asyncio.apply()


In [2]:

# DONE: Implement create_experience
class Experience:
    def __init__(self,workflow,round_added,score,parent_score=None,modification=None):
        self.workflow = workflow
        
        self.parent_workflow = self.workflow.parent
        if self.parent_workflow:
            self.parent_workflow_id = self.parent_workflow.id
        else:
            self.parent_workflow_id = None
        self.parent_score = parent_score
        
        if modification:
            self.round_modification_added = round_added
            self.round_added = self.parent_workflow.round_added
        else:
            self.round_modification_added = None
            self.round_added = round_added
        
        self.score = score
        self.modification = modification
        
        if self.parent_score and (isinstance(self.parent_score,float) or isinstance(self.parent_score,int)):
            self.success = self.score > self.parent_score
        else:
            self.success = None
    
    def __str__(self):
        return f"""
        Experience Info:
        Metadata:
        - Workflow with modification ID: {self.workflow.id}
        - Workflow with modification score: {self.score}
        - Workflow without modification ID: {self.parent_workflow_id}
        - Workflow without modification score: {self.parent_score}
        
        
        Modification that lead to this experience:
        - Did the modification lead to improvement over workflow without modifications: {self.success}
        - Round Workflow was modified: {self.round_added}
        - Modification details: {self.modification}
        """

# DONE: Implement Experience(Experiences?) obj in a way that makes retrieval by round easier
class Experiences:
    def __init__(self):
        self.experiences = {}
    
    def __getitem__(self, key):
        return self.experiences[key]
        
    def create_experience(self,workflow,round_,score,modification=None):
        if round_ not in self.experiences:
            self.experiences[round_] = {}
        parent_score = None
        if modification:
            #print(self.experiences,round_,workflow.parent.round_added,score,modification)
            parent_score = self.experiences[workflow.parent.round_added][workflow.parent.id].score
        self.experiences[round_][workflow.id] = Experience(workflow,round_,score,parent_score,modification)
        return self.experiences[round_][workflow.id]

In [3]:
class WorkflowNode:
    def __init__(self,description,prompt,modification,next_node=None):
        self.description = description
        self.prompt = prompt
        self.modification = modification
        if next_node is None:
            self.next = FinalNode()
        else:
            self.next = next_node()
        
    def __call__(self,**kwargs):
        #does something with prev_step and returns it
        llm_client = kwargs['llm_client']
        data = kwargs['data']
        
    def copy(self):
        return deepcopy(self)
    
class FinalNode:
    def __init__(self):
        self.next = None
        self.description = 'This node is a final sentinel and simply passes what it\'s given through.'
        
    async def __call__(self,**kwargs):
        #print(kwargs)
        return kwargs
    
    def copy(self):
        return self
    
class Workflow:
    #contains all of the information needed to understand a single workflow
    def __init__(self,round_added = 0,parent = None):
        self.next = FinalNode()
        self.round_added = round_added
        self.copies = 0
        # DONE: make sure workflows are being compared based on their IDs (since there may not be consistent parity between naive objects)
        self.id = str(uuid(NAMESPACE_DNS,str(self.round_added)+chr(97+self.copies)))
        self.parent = parent
        
    async def start(self,first_inputs):
        return await self.next(**first_inputs)
        
    def add_node(self,node_class:WorkflowNode,description,prompt,modification):
        new_node = node_class(description,prompt,modification)
        
        if not isinstance(self.next,FinalNode):
            current_node = self.next
            while not isinstance(current_node.next,FinalNode):
                current_node = current_node.next
            #print(type(current_node))
        else:
            current_node = self
        
        current_node.next = new_node
        return self
        
    def copy(self, round_):
        # Create a shallow copy of the current instance
        new_workflow = Workflow(round_,self)
        new_workflow.next = self.next.copy()
        new_workflow.copies = self.copies + 1
        # DONE: make sure workflows are being compared based on their IDs (since there may not be consistent parity between naive objects)
        new_workflow.id = str(uuid(NAMESPACE_DNS, str(new_workflow.round_added) + chr(97 + new_workflow.copies)))
        return new_workflow 

    def __lt__(self,other):
        return self.copies < other.copies
    
    def extract_chain_info(self):
        chain = []
        chain_info = []
        current_node = self.next
        while current_node.next:
            chain.append(type(current_node))
            chain_info.append(current_node.description)
            current_node = current_node.next
            
        return ' -->> '.join(map(str,chain)),'\n'.join([':'.join((node,node_info)) for node,node_info in zip(map(str,chain),chain_info)])

In [4]:
#available operators
# class GeneratePrompts(WorkflowNode):
#     def __init__():
#         super().__init__()
#     def __call__(self,**kwargs):
#         #does something with prev_step and returns it
#         llm_client = kwargs['llm_client']
#         data = kwargs['data']
#         prompt
#         results = []
#         for instance in data:
#             llm_client.chat.completions.create(
#                 messages=[
#                     messages+[
#                         {"role":"user"},
#                         {"content":instance}
#                     ]
#                 ],
#                 model='gpt-4o-mini'
#             ).messages[0].content
#         self.next(messages=results,data=data,
#                   llm_client=llm_client)        
#     def copy(self):
#         return deepcopy(self)
    
class GenerateText(WorkflowNode):
    def __init__(self,description,prompt,modification):
        super().__init__(description,prompt,modification)
        
    async def __call__(self,**kwargs):
        #does something with prev_step and returns it
        llm_client = kwargs.get('llm_client',None)
        data = kwargs.get('data',[])
        results = []
        #print('in node:',llm_client,';',data)
        for instance in data:
            result = (await llm_client.chat.completions.create(
                messages=[
                    {"role":"system","content":self.prompt},
                    {"role":"user","content":instance}
                ],
                model='gpt-4o-mini'
            )).choices[0].message.content
            results.append(result)
        return await self.next(data=results,llm_client=llm_client)
        
    def copy(self):
        return deepcopy(self)
    

In [5]:
# DONE: Implement split_data
def split_data(dataset,val_ratio=0.2,random_seed=1337):
    seed(random_seed)
    val_set = []
    test_set = []
    for datum in dataset:
        if random() < val_ratio:
            val_set.append(datum)
        else:
            test_set.append(datum)
            
    return val_set,test_set

In [6]:
# DONE: Implement determine_variance_threshold
# DONE: Implement select_high_variance_instances
# TODO: Handle asynchronous running (e.g. each output score set val data i may not correspond with input val data i)
def select_high_variance_instances(val_data,score_sets,marginal_variance_tolerance_perc = 0.05):
    variances = []
    n_data_points = len(score_sets[0])
    for i in range(n_data_points):
        scores_i = [score_set[i] for score_set in score_sets]
        num_scores_i = len(scores_i)
        avg_score_i = sum(scores_i)/num_scores_i
        sample_variance_i = (1/num_scores_i)*sum((score_i-avg_score_i)**2 for score_i in scores_i)
        variances.append(sample_variance_i)
        
    sorted_variances = sorted(list(set(variances)),reverse=True)
    #use elbow method diminishing gains based on marginal_variance_tolerance_perc
    last_variance = None
    for variance in sorted_variances:
        if last_variance is None:
            last_variance = variance
        else:
            perc_change = (variance/last_variance)-1
            if perc_change < marginal_variance_tolerance_perc:
                return last_variance
            last_variance = variance
            
    return [datum for i,datum in enumerate(val_data) if variances[i]>last_variance],last_variance
            

In [7]:
# DONE: Implement select_parent
def select_parent(topk_scores,topk_W):
    probabilities = calculate_mixed_probabilities(topk_scores) #DONE: Implement calculate_mixed_probabilities
    return sample_from_categorical(probabilities,topk_W) #DONE: Implement sample_from_categorical

In [8]:
#DONE: Implement calculate_mixed_probabilities
def calculate_mixed_probabilities(scores,lambda_=0.4,alpha=0.5): #TODO: Determine appropriate values of lambda and alpha from the paper
    n = len(scores)
    max_score = max(scores)
    w = list(e**(alpha*(s_i-max_score)) for s_i in scores)
    P_score = (w_i/sum(w) for w_i in w)
    P_uniform = (1/n for _ in range(n))
    P_mixed = (lambda_ * P_uniform_i + (1-lambda_) * P_score_i for P_score_i,P_uniform_i in zip(P_score,P_uniform))
    return P_mixed

In [9]:
#DONE: Implement sample_from_categorical
def sample_from_categorical(probabilities,workflows):
    return choices(population=workflows,weights=probabilities,k=1)[0]

In [10]:
#list(calculate_mixed_probabilities([0.33,0.233]))

In [11]:
# DONE: Implement load_context
def load_context(parent_workflow,experiences):
    #get the experiences related to parent_workflow
    #parent_workflow may or may not have modifications
    experience_info = []
    for round_ in experiences:
        if parent_workflow.id in experiences[round_]:
            experience_info.append(str(experiences[round_][parent_workflow.id]))
    workflow_experiences = '\n\n'.join(experience_info)
    
    #get the node chain visualization and node descriptions parent_workflow implements
    chain_viz,chain_descriptions = parent_workflow.extract_chain_info()
    
    context_string = f"""
        Context for workflow {parent_workflow.id}:
        
        Workflow node chain visualization:
        {chain_viz}
        
        Descriptions of each node:
        {chain_descriptions}
        
        Trials (experiences) involving this workflow:
        {workflow_experiences}
    """
    return context_string
    
            


In [12]:
# DONE: Implement optimize
class NodeAdd(BaseModel):
    node:str
    modification:str
    description:str
    prompt:str
    
async def optimize(round_,parent_workflow,context_string,dict_of_valid_operations,llm_client,reward_delay, parent_maturity):
    C,O = context_string,'\n\n'.join(map(str,dict_of_valid_operations.keys())) #TODO: Implement Node __str__ for this purpose
    
    optimize_system_prompt = f"""
    You are being given a chance to improve the workflow used to answer user queries.
    You will be given information about the workflows as well as information about, and results of, previous attempts (if applicable) to improve the workflow.
    You will also be given information about valid operations that can be added to the workflow in order to increase its ability to successfully answer the user query.
    
    You will be expected to reason step-by-step in order to come to a conclusion about what node (valid operation) to add to the workflow and the prompt that should direct it.
    
    At the end of your reasoning steps, you should output a single json object in the following format:
    {{"node":"<NAME OF NODE>","modification":"Add a new <NAME OF NODE> to <AND SO ON>","description":"<DESCRIPTION>","prompt:"<PROMPT>"}}
    where "<NAME OF NODE>" is one of given node names. If you fail to choose an existing node name, you will fail your task.
    And where <AND SO ON> is where the rest of your explanation should go. If you fail to rationally, simply and clearly describe 
    your reasoning here, you will fail your task and future optimizations will go poorly.
    Third, <DESCRIPTION> is your description of the node's purpose and how it is intended to go about its purpose. This is critical for communicating to yourself in the future about why you created a certain node, so be detailed.
    And, finally, <PROMPT>, which is where you exercise your best ability to optimize and improve the results of the next node. If you fail to rationally, simply, and clearly describe
    how the next node should interact with its data (which follows from the last node in the current workflow), the optimization task will fail utterly.

    Although your communication should be clear, you are allowed to use your resources creatively. For example, you might specify future node expansion if-then instructions in the descriptions,
    and/or you may instruct the model to keep part of the previous prompt in the answer so that it's available to later nodes.

    Be creative and err on the side of thinking out loud and saying something that isn't useful or doesn't get used rather than not having the idea at all.

    In addition, when the node is "GenerateText", you are allowed to use the node to generate any output you want, as long as it is within the limits of what is producible by an LLM alone.

    Although you may expect your changes to be evaluated at least {reward_delay} times (henceforth referred to as the reward delay) without critical failure, 
    unless you use nodes that capitalize on the information gained thus far, you will not receive a good evaluation, since your primary goal is to make the workflow fit for responding well within the system. 

    You have lived through {parent_maturity} evaluations and will die in {max(reward_delay-parent_maturity,0)} evaluations unless you improve.
    
    You are BRILLIANT OPTIMIZER.
    """
    
    optimize_user_prompt = f"""
    Here is your context:
    Workflow information:
    {C}
    
    Valid operations/nodes:
    {O}
    """
    
#     print(optimize_user_prompt)
    
    client_response = await llm_client.beta.chat.completions.parse(
        messages = [
            {"role":"system","content":optimize_system_prompt},
            {"role":"user"  ,"content":optimize_user_prompt}
        ],
        model='gpt-4o-mini',
        response_format=NodeAdd,
        temperature = 1.2
    )
    
    nodeadd_obj = client_response.choices[0].message.parsed
    
    #print(nodeadd_obj.dict())
    
    W_round = parent_workflow.copy(round_).add_node(dict_of_valid_operations.get(nodeadd_obj.node),nodeadd_obj.description,nodeadd_obj.prompt,nodeadd_obj.modification)
    return W_round,nodeadd_obj.modification

In [13]:
# na = NodeAdd(node = '1',description = '2',prompt = '3',modification = '4')
# # na.node = '1'
# # na.description = '2'
# # na.prompt = '3'
# # na.modification = '4'
# na.dict()

In [14]:
#DONE: Implement evaluator 
#will use gpt-4o to assess results (small dataset for now)
class Evaluation(BaseModel):
    grade:int#conint(ge=0,le=10)
        
async def evaluate(question,correct_answer,given_answer):
    evaluate_system_prompt = """
    You are being given a chance to evaluate the result of a workflow used to answer user queries.
    You will be given the user query, the correct answer, the the answer provided by the workflow.
    
    At the end of your succinct explanation, you should output only a single json object in the following format:
    {"grade":<GRADE>}
    where <GRADE> is an integer between 0 and 10, where 0 is completely irrelevant and/or completely unintelligible, and 10 is perfect in both content and understandability. In between is a continuum of integer grades describing differing levels of imperfection. 
    
    You are EVALUATOR.
    """
    
    evaluate_user_prompt = f"""
    Here is the user query:
    {question}
    
    Here is the correct answer:
    {correct_answer}
    
    Here is the answer generated by the workflow:
    {given_answer}
    """
    
    client_response = await llm_client.beta.chat.completions.parse(
        messages = [
            {"role":"system","content":evaluate_system_prompt},
            {"role":"user"  ,"content":evaluate_user_prompt}
        ],
        model='gpt-4o-mini',
        response_format=Evaluation,
        temperature = 0.0
    )
    
    evaluation_obj = client_response.choices[0].message.parsed
    return evaluation_obj.grade

In [15]:
# DONE: Implement execute_workflow_over_data
# TODO: Make this asynchronous
# TODO: Implement categorical loss
async def get_workflow_data_score(workflow,evaluator,datum,llm_client):
    q,a = datum
    a_hat = (await workflow.start({'data':[q],'llm_client':llm_client}))['data']
    score = await evaluator(q,a,a_hat)
    return score

async def execute_workflow_over_data(workflow,evaluator,data,llm_client):
    #scores = []
    #costs = []
    data_tasks = []
    for datum in data:
        data_tasks.append(get_workflow_data_score(workflow,evaluator,datum,llm_client))
         #this works for a continuous loss/gain (e.g. answer quality/lack of), TODO: Implement categorical loss
    scores = await asyncio.gather(*data_tasks)
        #costs.append(cost)
    return scores#,sum(costs)
        

In [16]:
async def get_round_scores(W_round,G,Dv,llm_client):
    scores = await execute_workflow_over_data(W_round,G,Dv,llm_client) #DONE: Implement execute_workflow_over_data
    round_score = sum(scores)/len(scores) #this works for a continuous loss/gain (e.g. answer quality/lack of), TODO: Implement categorical loss
    return round_score

In [17]:
async def get_optimal_workflow(initial_workflow,evaluator,dataset,operators,llm_client,number_of_rounds=20,k=3,early_stopping_rounds=5,scoring_rounds=5,reward_delay=3):
    #DONE: Implement evaluator
    #Initialize variables
    W_0, G, D, N, O, k, n, I = initial_workflow,evaluator,dataset,number_of_rounds,operators,k, \
                                early_stopping_rounds,scoring_rounds
    results = []
    experiences = Experiences()
    topk_W = []
    best_W = None
    topk_scores = []
    best_score = 0
    average_score = 0
    topk_W_unchanged = 0

    keep_alives = {}#{i:None for i in range(reward_delay)}
    
    #Split dataset
    if len(dataset)<50:
        Dv,Dt = dataset,dataset
    else:
        Dv,Dt = split_data(dataset) #DONE: Implement split_data
    
    #Gather initial scores to determine final validation dataset
    score_set_tasks = [execute_workflow_over_data(W_0,G,Dv,llm_client) for _ in range(k)] #DONE: Implement execute_workflow_over_data
    score_sets = asyncio.gather(*score_set_tasks)
    if len(Dv)>=50:
        Dv,variance_threshold = select_high_variance_instances(Dv,score_sets)   #DONE: Implement select_high_variance_instances
                                                                        #DONE: Implement determine_variance_threshold
    
        print(f'{len(Dv)} validation samples selected based on variance threshold {variance_threshold:.2f}')
    else:
        print(f'All {len(Dv)} validation samples selected for training')
    #MAIN LOOP----------------------------------------
    #Iterate to improve bestScore
    for round_ in (pbar := tqdm(range(N))):
        if round_ == 0:
            parent = W_0
        else:
            pbar.set_description(f'Best score:{best_score:.2f} Last round score:{average_score:.2f}')
            if len(topk_W) == 1:
                parent = topk_W[0]
            else:
                parent = select_parent(topk_scores,topk_W) #DONE: Implement select_parent
        
        #Load context for parent and perform optimization forward pass
        if round_>0:
            parent_maturity = [k for k,v in keep_alives.items() if parent==v[0]]
            if parent_maturity:
                parent_maturity = parent_maturity[0]
            else:
                parent_maturity = reward_delay + 1
            context = load_context(parent,experiences.experiences) #DONE: Implement load_context
            W_round, modification = await optimize(round_,parent,context,O,llm_client,reward_delay,parent_maturity) #DONE: Implement optimize
        else:
            W_round,modification = parent,''
        #Generate validation scores for modified workflow to demonstrate relative performance
        round_score_tasks=[]
        for i in range(I):
            round_score_tasks.append(get_round_scores(W_round,G,Dv,llm_client))
        round_scores = await asyncio.gather(*round_score_tasks)
        for round_score in round_scores:
            results.append((round_,round_score))
        
        #Capture a new experience for use in future optimization passes by using the modified workflow validation scores as gain
        round_scores = [r[1] for r in results if r[0]==round_]
        average_score = sum(round_scores)/len(round_scores)
        #print(average_score)
        experience = experiences.create_experience(W_round,round_,average_score,modification) #DONE: Implement create_experience
#         experiences.append(experience) #DONE: Implement Experience(Experiences?) obj in a way that makes retrieval by round easier
        
        #Save the previous topk_W for the later early-stopping check
        prev_topk_W = topk_W
        
        #Update best W, if applicable this round
        if average_score > best_score:
            best_score = average_score
            best_W = W_round

        if reward_delay in keep_alives:
            del keep_alives[reward_delay]
        keep_alives = {k+1:v for k,v in keep_alives.items()}
        keep_alives[0]=(W_round,average_score)

        #print(average_score,W_round.extract_chain_info())
        
        topk_W.append(W_round)
        topk_scores.append(average_score)
        
        #Try to include this round's W_round into topk_W if the W_round's score is within the top k scores
        combined = sorted(zip(topk_scores,topk_W),reverse=True)
        topk_scores[:],topk_W[:] = zip(*combined)
        
        #Boot any workflows outside of the top k scores
        #if len(topk_scores)>k:
        topk_scores = list(topk_scores[:k])
        topk_W = list(topk_W[:k])
        if round_!=N-1:
            topk_W += [keep_alives[k][0] for k in sorted(keep_alives)]
            topk_scores += [keep_alives[k][1] for k in sorted(keep_alives)]

        #make sure workflows delaying reward 
        
            
        #Check if topk_W is the same as the prev topk_W and increment the early stopping counter if so. If not, set counter to 0.
        if set([W.id for W in topk_W])==set([prevW.id for prevW in prev_topk_W]):
            topk_W_unchanged+=1 #DONE: make sure workflows are being compared based on their IDs 
                                                    #(since there may not be consistent parity between naive objects)
        else:
            topk_W_unchanged = 0
        
        #Stop the workflow improvement early if no new workflows have joined the top k workflows in n rounds
        if topk_W_unchanged >= n:
            break
    #END MAIN LOOP------------------------------------
    #return the best workflow and the k-1 next best workflows/the best workflow's score and the k-1 next best workflows' scores
    return (best_W,topk_W),(best_score,topk_scores),results,experiences
    

In [18]:
#!pip install openai --quiet
from openai import AsyncOpenAI

In [19]:
initial_workflow = Workflow()
operators = {'GenerateText':GenerateText} #registered nodes to be used in optimization
llm_client = AsyncOpenAI(api_key='')

In [20]:
qa_dataset = [
    ("Summarize the key themes discussed in a dataset of customer reviews about a new smartphone.", "The reviews primarily focus on the phone's excellent camera quality, long battery life, and sleek design. However, there are several complaints about the high price and lack of a headphone jack."),
    
    ("Perform sentiment analysis on the following dataset of tweets related to a recent product launch: 'This product is amazing, totally worth the money!', 'Disappointed with the quality. Expected more for the price.', 'Solid performance, but a bit overpriced.'", "The sentiments expressed in the dataset are mixed: one tweet is positive, one is negative, and the third expresses neutral sentiment about the product's price."),
    
    ("Translate the following French sentence into English: 'Le ciel est bleu.'", "The translation of 'Le ciel est bleu' is 'The sky is blue.'"),
    
    ("In a dataset of email conversations, identify if the following text is spam: 'Congratulations! You’ve won a free trip to the Bahamas. Click here to claim your prize.'", "The text is classified as spam based on the pattern of congratulatory messages and a suspicious link."),
    
    ("Extract all names of people from the following sentence: 'John and Mary went to Paris for a vacation.'", "The extracted names are: John, Mary."),
    
    ("Given a dataset of news articles, answer the following question: 'Who is the current president of the United States?' from the text: 'In 2024, Joe Biden is serving his second term as president.'", "The current president of the United States is Joe Biden."),
    
    # New examples
    ("Identify the sentiment of the following review: 'The hotel was beautiful but the service was terrible.'", "The sentiment is mixed, with positive feedback on the hotel itself but negative feedback regarding the service."),
    
    ("Given a dataset of product descriptions, extract the key features from this description: 'This laptop features a 15.6-inch display, Intel i7 processor, 16GB RAM, and a 512GB SSD.'", "The key features are: 15.6-inch display, Intel i7 processor, 16GB RAM, 512GB SSD."),
    
    ("Translate the following Spanish sentence into English: 'Me gusta mucho la comida mexicana.'", "The translation of 'Me gusta mucho la comida mexicana' is 'I really like Mexican food.'"),
    
    ("Classify the following text as either a question or a statement: 'What time does the movie start?'", "The text is classified as a question."),
    
    ("Based on the dataset of historical weather reports, answer the following: 'What was the highest temperature recorded in July 2020?'", "The highest temperature recorded in July 2020 was 104°F."),
    
    ("In a dataset of product reviews, identify if the following review is fake or genuine: 'This product is absolutely amazing and changed my life completely! I recommend it to everyone!'", "The review is likely fake due to its exaggerated and overly positive language.")
]


In [21]:
math_dataset = [
    ("What is the sum of 7 and 5?", "The sum of 7 and 5 is 12."),
    
    ("What is the product of 8 and 6?", "The product of 8 and 6 is 48."),
    
    ("If you subtract 9 from 15, what do you get?", "15 minus 9 equals 6."),
    
    ("What is the square root of 81?", "The square root of 81 is 9."),
    
    ("What is 25% of 200?", "25% of 200 is 50."),
    
    ("Solve for x: 5x = 20", "x equals 4."),
    
    ("If a rectangle has a length of 10 units and a width of 4 units, what is its area?", "The area of the rectangle is 40 square units."),
    
    ("What is the value of 2 raised to the power of 5?", "2 to the power of 5 is 32."),
    
    ("How many degrees are in a right angle?", "A right angle has 90 degrees."),
    
    ("If 12 is divided by 3, what is the quotient?", "The quotient is 4."),
    
    ("A car travels 60 miles in 2 hours. What is its average speed in miles per hour?", "The average speed is 30 miles per hour."),
    
    ("What is the perimeter of a square with a side length of 5 units?", "The perimeter is 20 units."),
    
    ("How many centimeters are there in 1 meter?", "There are 100 centimeters in 1 meter."),
    
    ("What is the greatest common divisor (GCD) of 24 and 36?", "The GCD of 24 and 36 is 12."),
    
    ("Convert 0.75 into a fraction in its simplest form.", "0.75 as a fraction in simplest form is 3/4."),
    
    ("If a triangle has angles of 60° and 70°, what is the measure of the third angle?", "The third angle measures 50°."),
    
    ("How many faces does a cube have?", "A cube has 6 faces."),
    
    ("Solve for y: 3y + 7 = 22", "y equals 5."),
    
    ("What is the average of the numbers 4, 8, and 12?", "The average is 8."),
    
    ("If you flip a fair coin, what is the probability of getting heads?", "The probability is 1/2.")
]


In [22]:
(W_star,topk_W),(best_score,topk_scores),results,experiences = await get_optimal_workflow(initial_workflow,evaluate,math_dataset,operators,llm_client,k=5,number_of_rounds=40)

All 20 validation samples selected for training


  0%|          | 0/40 [00:00<?, ?it/s]

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
topk_W

In [ ]:
best_nodes = []
best_w = topk_W[-1]
while best_w.next:
    best_nodes.append(best_w)
    best_w = best_w.next

In [ ]:
best_nodes

In [ ]:
[(node.__dict__) for node in best_nodes[1:]]

In [ ]:
# workflow_test = Workflow()
# workflow_test.add_node(GenerateText,'The GenerateText node is designed to create textual responses based on the input it receives. It can synthesize information, provide explanations, or generate creative content depending on the context of the user query. This node will enhance the workflow by ensuring that the responses are not only relevant but also articulated in a coherent and engaging manner.', 
#                        "Based on the user query, generate a detailed and informative text response that addresses the user's needs and provides additional context or examples where necessary.",
#                       'Add a new GenerateText to the workflow')
# print(workflow_test.extract_chain_info())
# execute_workflow_over_data(workflow_test,evaluate,dataset,llm_client)

### Future Enhancements
We can optimize one layer/level of operators.

What about two layers of operators? (e.g. operators nested in operators)

What about N layers of operators?


e.g. can we implement hierarchical optimization so that each and every step in the chain is tightly fit to purpose

e.g. can we construct operators from smaller units on the fly?

e.g. are there general operators such that these operators can be constructed of these operators at any level of depth


We have implemented breadth (operator columns) - can we implement depth (operator rows)?

Can we do it just as fast as breadth?